In [1]:
%matplotlib inline
from os import listdir
from os.path import isdir
from numpy import savez_compressed
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from numpy import load
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from time import perf_counter
import sys
import numpy as np
import io
import json
import math
import matplotlib.image as mpimg
from matplotlib import rcParams
import matplotlib.pyplot as plt
import ast
import tensorflow as tf
from numpy import expand_dims
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from scipy.spatial import distance
import scipy


In [2]:
def read_file():
    file = open("test_data.txt", "r")
    contents = file.read()
    dictionary = ast.literal_eval(contents)
    file.close()
    return dictionary

In [3]:
def find_images_matching_criteria(dictionary1, criteria):
    known_image = ''
    test_image = ''
    for key, value in dictionary1.items():
        if(key == criteria):
            known_image = value[0]
            test_image = value[1]
    known_image_path = 'known/'+known_image+'.jpg' 
    test_image_path = 'test/'+test_image+'.jpg'
    return known_image_path, test_image_path

In [4]:

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)

	print(len(results), 'faces have been found')

	#--------------------------- detecting the number of faces ------------------------------------
	if len(results) == 0:
		raise Exception('No faces detected')
	elif len(results) > 1:
		raise Exception('Multiple faces detected')		

	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
    
	return face_array


In [28]:
def get_embeddings(filenames):
    print("filenames------>", filenames)
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create predication model - note that we have several 
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat

In [ ]:
def plot_initial_images(path_known, path_test):
    
    rcParams['figure.figsize'] = 13 ,10
    img_A = mpimg.imread(path_known)
    img_B = mpimg.imread(path_test)
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(img_A);
    ax[1].imshow(img_B);

In [ ]:
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:	
		print('>face is a Match (%.3f <= %.3f) = %.0f' % (score, thresh, math.floor((1-score)*100)), "%")
	else:
		print('>face is NOT a Match (%.3f <= %.3f) = %.0f' % (score, thresh, math.floor((1-score)*100)), "%")

In [ ]:
def compare(image1, image2):
    print("-------images------", image1, image2)
    
    vector1 = get_embeddings([image1])[0]
    
    vector2 = get_embeddings([image2])[0]
    

    is_match(vector1, vector2)